# **Simple Agent & Tools:**

In [45]:
# install necessary libaries:

%pip install --upgrade --quiet sentence_transformers
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-google-genai langchain-chroma bs4 boto3
%pip install --upgrade --quiet langchain-aws

## **Let's Understand the Tools:**


Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:<br>

1. The name of the tool
2. A description of what the tool is
3. JSON schema of what the inputs to the tool are
The function to call
4. Whether the result of a tool should be returned directly to the user.
<br>

It is useful to have all this information because this information can be used to build action-taking systems! The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take, and then the function to call is equivalent to taking that action.<br>

The simpler the input to a tool is, the easier it is for an LLM to be able to use it. Many agents will only work with tools that have a single string input. For a list of agent types and which ones work with more complicated inputs, please see this documentation

<br>
Importantly, the name, description, and JSON schema (if used) are all used in the prompt. Therefore, it is really important that they are clear and describe exactly how the tool should be used. You may need to change the default name, description, or JSON schema if the LLM is not understanding how to use the tool.



### **WikipediaAPIWrapper Tool:**

In [6]:
%pip install wikipedia

In [7]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [17]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [18]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [10]:
tool.name

'wikipedia'

In [11]:
# tool descriptions:

tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [12]:
# args:

tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [19]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [20]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of '

### **YouTubeSearchTool:**

In [ ]:
%pip install youtube-search

In [24]:
from langchain_community.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

In [22]:
dct = {
    "Tool Name":tool.name,
    "Tool Description": tool.description,
    "Tool Args": tool.args
}

dct

{'Tool Name': 'youtube_search',
 'Tool Description': 'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional',
 'Tool Args': {'query': {'title': 'Query', 'type': 'string'}}}

In [27]:
tool.run("Krish Naik")

"['https://www.youtube.com/watch?v=JxgmHe2NyeY&pp=ygUKS3Jpc2ggTmFpaw%3D%3D', 'https://www.youtube.com/watch?v=d2kxUVwWWwU&pp=ygUKS3Jpc2ggTmFpaw%3D%3D']"

### **TavilySearchResults:**

In [ ]:
%pip install tavily-python

In [38]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os

tavily_api_key = "*******************"

os.environ["TAVILY_API_KEY"] = tavily_api_key

search = TavilySearchResults()

In [39]:
search.invoke("what is the weather in Siliguri, West Bengal Now.")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Siliguri', 'region': 'West Bengal', 'country': 'India', 'lat': 26.7161, 'lon': 88.4236, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1734514177, 'localtime': '2024-12-18 14:59'}, 'current': {'last_updated_epoch': 1734513300, 'last_updated': '2024-12-18 14:45', 'temp_c': 25.1, 'temp_f': 77.1, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 158, 'wind_dir': 'SSE', 'pressure_mb': 1012.0, 'pressure_in': 29.87, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 25, 'cloud': 0, 'feelslike_c': 24.4, 'feelslike_f': 75.8, 'windchill_c': 25.1, 'windchill_f': 77.1, 'heatindex_c': 24.4, 'heatindex_f': 75.8, 'dewpoint_c': 3.8, 'dewpoint_f': 38.8, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.4, 'gust_mph': 4.4, 'gust_kph': 7.1}}"},
 {'url': 'https://www.worldweatheronline.com/siliguri-weather/west-bengal/in.aspx',


### **Create Custom Tools:**

In [40]:
from langchain.agents import tool

In [41]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [42]:
# Invoking:

get_word_length.invoke("abc")

3

In [44]:
# Get the Details of Custom Tools:

print("Multiplication Tool:")
print("Tool Name: ", multiply.name)
print("Tool Description: ", multiply.description)
print("Tool Arguments: ", multiply.args)

Multiplication Tool:
Tool Name:  multiply
Tool Description:  Multiply two numbers.
Tool Arguments:  {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## **Agent & LLM Integration:**

In [46]:
# Load the Tokens:

from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

### **Load LLM:**

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

print(llm.invoke("hi").content)

Hi there! How can I help you today?



### **Load Embeddings from HF:**

In [ ]:
# Get the Embeddings:

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

### **Agnt & LLM:**

In [51]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [53]:
# Call/Create Tool:

tool = load_tools(["wikipedia"], llm=llm)

In [54]:
# Initiaize Agent:

agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

<ipython-input-54-a4a172ab75be>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [55]:
# Agent Invoke:

agent.invoke("can you tell me what is current GDP of india?")



> Entering new AgentExecutor chain...
Thought: I need to search Wikipedia for India's current GDP.

Action: wikipedia
Action Input: India GDP

Observation: Page: List of Indian states and union territories by GDP
Summary: These are lists of Indian states and union territories by their nominal gross state domestic product (GSDP). GSDP is the sum of all value added by industries within each state or union territory and serves as a counterpart to the national gross domestic product (GDP). As of 2011, the Government accounted for about 21% of the GDP followed by agriculture with 21% and corporate sector at 12%. The remaining 48% is sourced from small proprietorship and partnership companies, unorganized sector and households.



Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fifth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita in

{'input': 'can you tell me what is current GDP of india?',
 'output': 'I cannot provide the exact current GDP of India using only Wikipedia.  Please consult a dedicated financial data source like the IMF or a reputable financial news outlet for this information.'}